In [ ]:
import torch
import random
import numpy as np

In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [ ]:
import torchvision.datasets # MNIST - images of hand mage numbers of 0-9
MNIST_train = torchvision.datasets.MNIST('./', download=True, train=True)
MNIST_test = torchvision.datasets.MNIST('./', download=True, train=False)

In [ ]:
X_train = MNIST_train.train_data
y_train = MNIST_train.train_labels
X_test = MNIST_test.test_data
y_test = MNIST_test.test_labels

In [ ]:
X_train.dtype, y_train.dtype
#x- unsigneed int 8, y - int64

In [ ]:
X_train = X_train.float()
X_test = X_test.float()

In [ ]:
X_train.shape, X_test.shape
# 60 000 images of 28x28

In [ ]:
y_train.shape, y_test.shape

In [ ]:
#here we can see the image
import matplotlib.pyplot as plt
plt.imshow(X_train[0, :, :])
plt.show()
print(y_train[0])

In [ ]:
#растянем каждое изображение в вектор, будет 60 000 изображенией но уже не 28х28 а 1х28**2 
X_train = X_train.reshape([-1, 28 * 28])
X_test = X_test.reshape([-1, 28 * 28])

In [ ]:
class MNISTNet(torch.nn.Module):
    def __init__(self, n_hidden_neurons):
        super(MNISTNet, self).__init__()
        self.fc1 = torch.nn.Linear(28 * 28, n_hidden_neurons)
        self.ac1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_hidden_neurons, 10)#тк всего 10 классво 0-9 
    
    def forward(self, x):#пропускает Х через все слои последовательно
        x = self.fc1(x)
        x = self.ac1(x)
        x = self.fc2(x)
        return x
    
mnist_net = MNISTNet(100)

In [ ]:
# torch.cuda.is_available()

In [ ]:
# !nvidia-smi

In [ ]:
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# mnist_net = mnist_net.to(device)
# list(mnist_net.parameters())

In [ ]:
#на вход пренимает НЕ вер-ти,а те выходы что до софтмакса
loss = torch.nn.CrossEntropyLoss()
#на вход адаму передаются все параметры нейронной сети,- оптимизируем веса,хранящиеся в нейронах
optimizer = torch.optim.Adam(mnist_net.parameters(), lr=1.0e-3)

## Убедиться что ран_тайм стоит на ГПУ а не на ЦПУ

### проверка видимости ГПУ библ-кой пай_торч
#### torch.cuda.is_available()
 
### проверка занятости карты какими-либо вычислениями (консольная команда)
#### !nvidia-smi  

### переолжим тензоры на видеокарту: веса нейрон сети, входы-картинки.
#### нулевую видюкарту выбираем ес их много, если нет вид.хи то на ЦПУ:
#### device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

#### переложим на девайс нейрон сеть:
#### mnist_net = mnist_net.to(device)

#### проверка - переложились ли:
#### list(mnist_net.parameters())

In [ ]:
#дела=им наш датасет на ,,маленькие,, части - батчи,
batch_size = 100

# test_accuracy_history = []
# test_loss_history = []

# X_test = X_test.to(device)
# y_test = y_test.to(device)

for epoch in range(10000):
    #перемешиваем датасет на каждой эпохе,выделять оттуда последовательные участки
    #т.о чтобы внутри 1 1эпохи каждый элемент/картинка была показана 1 раз
    order = np.random.permutation(len(X_train))
    
    for start_index in range(0, len(X_train), batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        
        X_batch = X_train[batch_indexes] #.to(device)
        y_batch = y_train[batch_indexes] #.to(device)
        
        preds = mnist_net.forward(X_batch) 
        
        #считаем лосс-ф по нашим предикшанам и сравниваем с заготовленными лейблами
        loss_value = loss(preds, y_batch)
        loss_value.backward()#считаем градиенты
        
        optimizer.step()#делаем градиентный шаг

    test_preds = mnist_net.forward(X_test)
#   test_loss_history.append(loss(test_preds, y_test))

    #предсказание класса(с макс. выходом) и сравниваем с реальным классом == 0-9
    #считаем среднее кол-во угадываний
    accuracy = (test_preds.argmax(dim=1) == y_test).float().mean()
    #test_accuracy_history.append(accuracy)
    print(accuracy)


In [ ]:
plt.plot(test_accuracy_history)
plt.plot(test_loss_history);